In [ ]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn import metrics
import matplotlib.pyplot as plt
import csv

## Neural Network

In [ ]:
def read_data(file_name: str) -> list[int]:
    labels = []
    attributes = []
    with open(file_name) as file:
        csv_reader = csv.reader(file)
        index_of = {name: index for index, name in enumerate(next(csv_reader))}
        # attributes = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'day_of_week', 'campaign', 'previous', 'poutcome']
        for line in csv_reader:
            label = int(line[index_of['y']] == 'yes')
            labels.append(label)
            attributes.append(clean_example(index_of, line))
    return labels, attributes

def clean_example(index_of: dict[str:int], line: list[str]) -> list[int]:
    jobs = ["admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown"]
    marital = ["divorced","married","single","unknown"]
    education = ["basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown"]
    contact = ["cellular","telephone"]
    day_of_week = ["mon","tue","wed","thu","fri"]
    poutcome = ["failure","nonexistent","success"]
    binary = ["no", "unknown", "yes"]
    age_min, age_max = 17, 98
    age_numeric = (int(line[index_of['age']]) - age_min) / (age_max - age_min)
    jobs_numeric = {job: index/(len(jobs)-1) for index, job in enumerate(jobs)}
    marital_numeric = {status: index/(len(marital)-1) for index, status in enumerate(marital)}
    education_numeric = {degree: index/(len(education)-1) for index, degree in enumerate(education)}
    default_numeric = {status: index/(len(binary)-1) for index, status in enumerate(binary)}
    housing_numeric = default_numeric
    loan_numeric = default_numeric
    contact_numeric = {method: index/(len(contact)-1) for index, method in enumerate(contact)}
    day_of_week_numeric = {day: index/(len(day_of_week)-1) for index, day in enumerate(day_of_week)}
    campaign_min, campaign_max = 1, 56
    campaign_numeric = (int(line[index_of['campaign']]) - campaign_min) / (campaign_max - campaign_min)
    previous_min, previous_max = 0, 7
    previous_numeric = (int(line[index_of['previous']]) - previous_min) / (previous_max - previous_min)
    poutcome_numeric = {outcome: index/(len(poutcome)-1) for index, outcome in enumerate(poutcome)}
    return [age_numeric, jobs_numeric[line[index_of['job']]], marital_numeric[line[index_of['marital']]], education_numeric[line[index_of['education']]], default_numeric[line[index_of['default']]], housing_numeric[line[index_of['housing']]], loan_numeric[line[index_of['loan']]], contact_numeric[line[index_of['contact']]], day_of_week_numeric[line[index_of['day_of_week']]], campaign_numeric, previous_numeric, poutcome_numeric[line[index_of['poutcome']]]]

class FeedForward(nn.Module):
    def __init__(self) -> None:
        super(FeedForward, self).__init__()
        self.relu = nn.LeakyReLU()
        self.linear1 = nn.Linear(12, 32)
        self.linear2 = nn.Linear(32, 16)
        self.out = nn.Linear(16, 2)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.out(x)
        return x

def neural_network() -> None:
    # Read data
    train_labels, train_data = read_data('bank-train.csv')
    valid_labels, valid_data = read_data('bank-valid.csv')
    test_labels, test_data = read_data('bank-test.csv')

    train_dataset = TensorDataset(torch.tensor(train_data), torch.tensor(train_labels))
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    valid_dataset = TensorDataset(torch.tensor(valid_data), torch.tensor(valid_labels))
    valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

    # Initialize model
    ff = FeedForward()
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(ff.parameters(), lr=0.01)

    # Train the model
    ff.train()
    train_losses = []
    valid_losses = []
    for _ in range(20):
        train_loss = 0
        valid_loss = 0
        for x, y in train_loader:
            optimizer.zero_grad()
            y_hat = ff(x)
            loss_val = loss(y_hat, y)
            train_loss += loss_val.item()
            loss_val.backward()
            optimizer.step()
        train_losses.append(train_loss)
        for x, y in valid_loader:
            y_hat = ff(x)
            loss_val = loss(y_hat, y)
            valid_loss += loss_val.item()
        valid_losses.append(valid_loss)

    # Accuracy of test data
    with torch.no_grad():
        ff.eval()
        predictions = ff(torch.tensor(test_data))
        labels = [torch.argmax(prediction) for prediction in predictions]
        correct_list = [int(label == prediction) for label, prediction in zip(test_labels, labels)]
        accuracy = sum(correct_list) / len(correct_list)
    print(f"Accuracy: {accuracy:.2%}")

    # Plot losses
    plt.plot([i for i in range(len(train_losses))], train_losses, label="Train Loss")
    plt.title("Train Loss vs. Epoch")
    plt.figure()
    plt.plot([i for i in range(len(valid_losses))], valid_losses, label="Validation Loss")
    plt.title("Validation Loss vs. Epoch")
    matrix = metrics.confusion_matrix(test_labels, labels)
    matrix_display = metrics.ConfusionMatrixDisplay(matrix)
    matrix_display.plot()
    plt.show()

neural_network()

## Decision Trees

In [144]:
from collections import Counter
from math import log2
from random import shuffle

class Node:
  def __init__(self, label:int=None, children:dict=None):
    self.label = label
    if children == None:
      self.children = dict()
    else:
      self.children = children

def are_all_same(examples: list[dict]) -> str|None:
    """Returns the class of a dataset if each example has the same class; otherwise, it returns nothing

    Args:
        examples (list[dict]): An array of examples. Each example is a dictionary of attribute:value pairs,
    and the target class variable is a special attribute with the name "Class"

    Returns:
        str|None: Class or None
    """
    first_class = examples[0]['y']
    for example in examples:
        if example['y'] != first_class:
            return None
    return first_class

def most_common_class(examples: list[dict], target: str='y') -> str:
    """Returns the most common class of the dataset

    Args:
        examples (list[dict]): An array of examples. Each example is a dictionary of attribute:value pairs,
    and the target class variable is a special attribute with the name "Class"
        target (str, optional): Defaults to 'Class'.

    Returns:
        str: The most common class
    """
    classes: Counter = Counter([example[target] for example in examples])
    return classes.most_common(1)[0][0]

def min_entropy(examples: list[dict], attributes: dict[str:set]) -> tuple[str,float]:
  """Returns a tuple of (attribute, entropy) for the attribute with the highest information gain

  Args:
      examples (list[dict]): An array of examples. Each example is a dictionary of attribute:value pairs,
  and the target class variable is a special attribute with the name "Class"
      attributes (dict[float:set]): A dictionary of attribute:values pairs

  Returns:
      tuple[str, float]: tuple of (attribute, info gain of attribute)
  """
  def entopry(attribute: str) -> float:
    value_to_count = {value:[] for value in attributes[attribute]}

    # initializing probabilities of value
    for example in examples:
      value_to_count[example[attribute]].append(example)

    # for each value of attribute, finding starting_entropy of data_value ⊆ data
    starting_entropy = 0
    for value_examples in value_to_count.values():
      # value_examples = [{Class: 1, Weather: Cold}, {Class: 0, Weather: Cold}]
      value_count = len(value_examples)
      prob_val = value_count/len(examples)
      class_to_count = {}
      # initializing class count for each value
      for ex in value_examples:
        # ex = {Class: 1, Weather: Cold}
        ex_class = ex['y']
        if class_to_count.get(ex_class, False):
          class_to_count[ex_class] += 1
        else:
          class_to_count[ex_class] = 1
      
      val_h = 0
      for class_count in class_to_count.values():
        prob_class_in_value = class_count/value_count
        val_h += -1*(prob_class_in_value) * log2(prob_class_in_value)
      starting_entropy += prob_val * val_h

    return starting_entropy
  
  attr_entropy_list = []
  for attribute in attributes:
    attr_entropy_list.append((attribute, entopry(attribute)))
  
  return min(attr_entropy_list, key=lambda x: x[1])

def read_data_tree(file_name: str) -> list[dict]:
    data = []
    with open(file_name) as file:
        csv_reader = csv.reader(file)
        care_about = {'y', 'age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'day_of_week', 'campaign', 'previous', 'poutcome'}
        index_of = {name: index for index, name in enumerate(next(csv_reader))}
        for line in csv_reader:
            line = {name: line[index] for name, index in index_of.items() if name in care_about}
            line['age'] = str(int(line['age']) - int(line['age']) % 20)
            line['campaign'] = str(int(line['campaign']) - int(line['campaign']) % 10)
            data.append(line)
    return data

def decision_tree(examples: list) -> Node:
    attrs_vals = {
        "age": {"0", "20", "40", "60", "80"},
        "job": {"admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown"},
        "marital": {"divorced","married","single","unknown"},
        "education": {"basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown"},
        "default": {"no", "unknown", "yes"},
        "housing": {"no", "unknown", "yes"},
        "loan": {"no", "unknown", "yes"},
        "contact": {"cellular","telephone"},
        "day_of_week": {"mon","tue","wed","thu","fri"},
        "campaign": {"0", "10", "20", "30", "40", "50"},
        "previous": {"0", "1", "2", "3", "4", "5", "6", "7"},
        "poutcome": {"failure","nonexistent","success"}
    }
    def recurse(data: list=examples, attributes: list=attrs_vals) -> Node:
        # Bases cases
        # !None = all same
        if are_all_same(data) != None:
            return Node(are_all_same(data))
        node = Node(most_common_class(data))
        if not attributes:
            return node
    
        # Recursive cases
        best_attribute = min_entropy(data, attributes)[0] # [0] is the attribute name
        node.label = best_attribute

        # Attach a default node in case an attribute is missing
        # node.children['unknown'] = most_common_class(data)

        # attributes[best_attribute] is a set of all possible values for the best attribute
        for value in attributes[best_attribute]:
            value_data = [d for d in data if d[best_attribute] == value] # list[dict]
            if not value_data:
               value_node = Node(most_common_class(data))
               node.children[value] = value_node
               continue

            value_attributes = {attribute:values.copy() for attribute, values in attributes.items() if attribute != best_attribute}
            value_node = recurse(value_data, value_attributes)
            node.children[value] = value_node
        
        return node

    return recurse()

def evaluate(tree: Node, example: dict) -> str:
    attribute = tree.label
    if not tree.children:
        return attribute
    value = example[attribute]
    next_tree = tree.children[value]
    if not next_tree:
        return tree.children['unknown']
    return evaluate(next_tree, example)

def most_common_class_of_node(node: Node) -> str:
  """Returns the most common class a node returns

  Args:
      node (Node): A node

  Returns:
      str: A class
  """
  class_to_count = {}

  def recurse(tree: Node=node) -> None:
    # base case 
    if not tree.children:
      class_ = tree.label
      if not class_to_count.get(class_, False):
        class_to_count[class_] = 1
      else:
        class_to_count[class_] += 1
      return
    for attribute, subtree in tree.children.items():
      if attribute != 'default':
        recurse(subtree)

  recurse()
  
  return max(class_to_count, key=lambda x: class_to_count[x])

def prune(node: Node, examples: list[dict]) -> None:
  """Takes in a trained tree and a validation set of examples.  Prunes nodes in order
  to improve accuracy on the validation data; the precise pruning strategy is up to you.

  Args:
      node (Node): a trained tree
      examples (list[dict]): a validation set of examples
  """
  # reduced error pruning
  def recurse(tree: Node=node) -> None:
    accuracy = 0
    for example in examples:
        label = example['y']
        y_hat = evaluate(tree, example)
        accuracy += int(label == y_hat)
    accuracy = accuracy / len(examples)
    for value, subtree in tree.children.items():
      # base cases
      if value == 'default':
        continue
      if not subtree.children:
        continue

      # change node to most common class
      tree.children[value] = Node(most_common_class_of_node(subtree))
      new_accuracy = 0
      for example in examples:
        label = example['y']
        y_hat = evaluate(tree, example)
        new_accuracy += int(label == y_hat)
      new_accuracy = new_accuracy / len(examples)

      # put node back if worse, then recurse on node
      if new_accuracy < accuracy:
        tree.children[value] = subtree
        recurse(subtree)
      # if we keep pruned node, continue to next child
  recurse()

# Read data
data = read_data_tree('bank-train.csv')
shuffle(data)
size = int(0.9 * len(data))
train_data = data[:size]
test_data = data[size:]

# Train the model
tree = decision_tree(train_data)

# Test before pruning
print("Accuracy before pruning:")
correct = 0
for example in test_data:
    label = example['y']
    y_hat = evaluate(tree, example)
    correct += int(label == y_hat)
accuracy = correct / len(test_data)
print(f"Test accuracy: {accuracy:.2%}")
correct = 0
for example in train_data:
    label = example['y']
    y_hat = evaluate(tree, example)
    correct += int(label == y_hat)
accuracy = correct / len(train_data)
print(f"Train accuracy: {accuracy:.2%}")

# Test after pruning
print("\nAccuracy after pruning:")
prune(tree, test_data)
correct = 0
for example in test_data:
    label = example['y']
    y_hat = evaluate(tree, example)
    correct += int(label == y_hat)
accuracy = correct / len(test_data)
print(f"Test accuracy: {accuracy:.2%}")
correct = 0
for example in train_data:
    label = example['y']
    y_hat = evaluate(tree, example)
    correct += int(label == y_hat)
accuracy = correct / len(train_data)
print(f"Train accuracy: {accuracy:.2%}")

Accuracy before pruning:
Test accuracy: 85.99%
Train accuracy: 92.98%

Accuracy after pruning:
Test accuracy: 89.24%
Train accuracy: 89.80%
